.

<span style="color: #8B008B; font-weight: bold; font-family: Arial; font-size: 3em;">Sistema de recomendación de librerías Python</span>

 <span style="color:#4B0082; font-family: Arial; font-size: 2em;">José Pérez Yázquez</span>

 <span style="color:#4B0082; font-family: Arial; font-size: 1.5em;">Octubre de 2017</span>

---

<span style="color: #12297c; font-family: Arial; font-size: 3em;">Introducción</span>

El presente proyecto implementa una propuesta de recomendador de librerías Python, la idea es que dado un proyecto que estemos desarrollando, nos recomiende librerías que otros proyectos similares han usado. Para ello usa como "corpus" la BBDD por excelencia de código, **Github**, y como medida de similitud sus descripciones y las librerías que ya compartan.

El proyecto consta de dos fases, en la primera de ellas obtendremos todos los metadatos de los proyectos usando la API Rest de Github y en la segunda parte, basándonos en esos metadatos construiremos el sistema de recomendación.

Aunque el objetivo del proyecto es procesar proyectos **Python**, sería fácilmente extensible para que procesara proyectos implementados en otros lenguajes. Por ejemplo, si quisiéramos hacer un sistema de recomendación de librerías **R**, solo sería necesario modificar la expresión regular que busca los **import** para que buscase la instrucción **library**.



        

.

<span style="color: #12297c; font-family: Arial; font-size: 3em;">Carga y procesamiento de datos</span>        

Esta fase se divide a su vez en cuatro subfases:
 - Recopilación de la información de los proyectos
 - Clonado de los proyectos
 - Procesamiento de los proyectos clonados
 - Preprocesamiento de los datos

# Recopilación de la información de los proyectos

Usando la API Rest de Github descarga la información, metadatos, de todos los proyectos que cumplen una serie de criterios, estos serán los siguientes:
  - Identificados como proyectos Python
  - Creado entre el 1 de Enero de 2012 y el 31 de Octubre de 2017
  - Con un mínimo de 10 estrellas

Toda la información es almacenada en MongoDB concretamente en una colección llamada **projects** de una BD llamada **github**, la información almacenada es la siguiente:
 - **id**: Identificador generado por Github
 - **name**: Nombre del proyecto
 - **full_name**: Nombre ampliado del proyecto
 - **created_at**: Fecha de creación
 - **git_url**: Url al repositorio (es lo que se usa cuando se quiere clonar)
 - **description**: Descripción del proyecto
 - **language**: Lenguaje de programación del proyecto (Java, Python, Scala, etc)
 - **stars**: Número de estrellas del proyecto, nos da una medida de la *populariodad* del proyecto
 - **library**: Lista con las librerías que el proyecto usa, en principio estará vacía y se alimentará cuando se procese el proyecto
 - **readme_txt**: Almacenará el texto de los ficheros README de los proyectos.
 - **readme_language**: Indica el idioma en el que está redactado el fichero README
 - **readme_words**: Descomposición del fichero README en palabras con significado semántico, será la información base para el sistema de recomendación
 - **raw_data**: Almacena toda la información del proyecto sin procesar por si más adelante es necesario algún dato que en estos momentos no parece relevante.
 
Además se incluirán un par de propiedades de uso interno

 - **library_lines**: Lineas de los ficheros **py** en los que se detectó la importación de una librería. De uso interno
 - **pipeline_status**: Algunas de las operaciones que se realizan pueden ser muy costosas en cuanto al tiempo que consumen, por ello se lleva para cada proyecto el registro del estado en el que está. 

Cuando se usa la API de Github sin autenticación existen una serie de límites que no podemos superar, el primero es que el número máximo de resultados que cualquier consulta puede devolver
es de 1000, para salvar este impedimento, vamos a realizar sucesivas consultas restringiendo las llamadas a un solo día del intervalo que vamos a cubrir.

Por otra parte, existe otra restricción en el uso de la API, el número de llamadas por minuto está limitado a 60, podríamos incrementar esta cantidad si las llamadas son autenticadas,
esto es usar por ejemplo un "client ID" y "secret"  como partes de la consulta, pero vamos abordar una estrategia diferente. Se irán realizando las consultas sin atender a los límites y cuando ese límite se supere, se suspenderá el proceso durante 61 segundos y posteriormente se reanudará
por el punto en el que iba.

Si se desea consultar los límites comentados, podemos hacer una consulta, por ejemplo usando curl

curl -i https://api.github.com/users/octocat

HTTP/1.1 200 OK

Date: Mon, 01 Jul 2017 17:27:06 GMT

Status: 200 OK

X-RateLimit-Limit: 60

X-RateLimit-Remaining: 56

X-RateLimit-Reset: 1372700873


En la cabecera de la respuesta nos muestra los límites con el valor actual:

 - **X-RateLimit-Limit**       Número máximo de solicitudes que se le permite hacer por hora.
 - **X-RateLimit-Remaining**   Número de solicitudes restantes en la ventana de límite de velocidad actual.
 - **X-RateLimit-Reset**       Hora a la que se restablece la ventana de límite de velocidad actual en segundos UTC.

En primer lugar vamos a cargar las librerías necesarias para esta fase del proyecto

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [2]:
from pymongo import MongoClient
from datetime import date, timedelta
from time import sleep
import sys
import os
import re
import json
import requests

A continuación vamos a definir una serie de constantes que serán usadas en el código, de esta manera podremos cambiar facilimente alguno de estos valores.
El significado de cada una de ellas es el siguiente:
 - **MIN_STARTS**: Número mínimo de estrellas que tiene que tener un proyecto para ser considerado
 - **START_DATE**: Fecha de inicio del intervalo, haciendo referencia a la fecha de creación del proyecto en Github
 - **END_DATE**: Fecha de fin del intervalo
 - **URL_PATTERN**: Patrón de la URL para la consulta a la API de Github, podemos ver que filtramos por lenguaje (Python), por fecha de creación (la suministraremos durante la ejecución) y por numero de estrellas (referido a la constante comentada anteriormente).
 - **ROOT_PATH**: Directorio raiz donde clonaremos los repositorios (debe existir antes de empezar a clonar los proyectos).
 - **CLONE_COMMAND**: Comando usado para clonar los repositorios
 - **LIBRARY_PATTERN**: Patrón que cumplen las líneas de código en las que se importa una librería
 - **FILE_LANGUAGE_EXTENSION**: Extensión de los ficheros para el lenguaje a procesar.

En la introducción deciamos que el sistema podrías ser extensible a otros lenguajes más allá de Python, para ello tendríamos que cambiar las dos últimas constantes a las apropiadas al lenguaje que queramos procesar, por ejemplo para **Java**, la última constante valdría **".java"** y para **R** valdría **".r"**

In [3]:
MIN_STARTS = 10
START_DATE = date(2017, 8, 1)
END_DATE = date(2017, 8, 2)
URL_PATTERN = 'https://api.github.com/search/repositories?q=language:Python+created:{0}+stars:>={1}&type=Repositories'

CLONE_COMMAND = "git clone {0} {1}"
ROOT_PATH = "d:/tfm/tmp"

LIBRARY_PATTERN = '(?m)^(?:from[ ]+(\S+)[ ]+)?import[ ]+([\S,\s]+)(\n)$'
FILE_LANGUAGE_EXTENSION = ".py"

Como se ha comentado anteriormente, vamos a usar MongoDB como tecnología de persistencia, vamos a definir una funcion que nos devuelva una referencia a la colección de proyectos con la que estamos trabajando.

In [4]:
def get_repository_projects():
    client = MongoClient('localhost', 27017)
    db = client.github_t5
    return db.projects

La siguiente función es la que realmente hace las llamadas a la API, tendrá como entrada la fecha de creación de los proyectos, usará la URL que hemos definicdo como constante.

In [5]:
def get_projects_by_date(date):
    print("Processing date", date)
    url_pattern = URL_PATTERN
    url = url_pattern.format(date, MIN_STARTS)
    response = requests.get(url)
    if (response.ok):
        response_data = json.loads(response.content.decode('utf-8'))['items']
        for project in response_data:
            insert_project(project)
    else:
        response.raise_for_status()

A continuanción definimos otra función auxiliar, que dada la información de la respuesta de la API, seleccionamos los atributos que vamos a necesitar y los almacenamos como un documento Mongo. Adicionalmente crea las propiedades mencionadas previamente (readme_txt,library,etc.).

In [6]:
def insert_project(github_project):
    if repository_projects.find_one({"id": github_project["id"]}):
        print("Project {0} is already included in the repository".format(github_project["name"]))
    else:
        project = {
            'id': github_project["id"],
            'name': github_project["name"],
            'full_name': github_project["full_name"],
            'created_at': github_project["created_at"],
            'git_url': github_project["git_url"],
            'description': github_project["description"],
            'language': github_project["language"],
            'stars': github_project["stargazers_count"],
            'readme_txt': "",
            'readme_language': None,
            'readme_words': [],
            'library': [],            
            'raw_data': github_project,
            'pipeline_status': 'INITIAL',
            'library_lines': [],
        }
        repository_projects.insert(project)

Finalmente ejecutamos el programa que, usando las funciones anteriormente definidas, descarga la información de los proyectos y la inserta en la BBDD, podemos observar que descompone las llamadas para traer en cada una de ellas solo los proyectos de un determinado dia y que *"gestiona"* las restricciones que tenemos en cuanto a llamada en la ventana de tiempo actual. 

En primer lugar recuperamos la colección en la que vamos a insertar los proyectos.

In [7]:
repository_projects = get_repository_projects()

Cargamos los proyectos en la colección, aplicamos las técnicas descritas para salvar las restricciones que nos impone la API de Github (número de items devuelto por cada consulta y número de llamadas por minuto).

In [8]:
for project_create_at in [START_DATE + timedelta(days=x) for x in range((END_DATE - START_DATE).days + 1)]:
    try:
        get_projects_by_date(project_create_at)
    except:
        print(">> Reached call limit, waiting 61 seconds...")
        sleep(61)
        get_projects_by_date(project_create_at)

Processing date 2017-08-01
Processing date 2017-08-02


En este punto tendriamos cargada toda la información que necesitamos de Github.

# Clonado de los proyectos

Para analizar las librerías que un proyecto utiliza, un camino podría ser buscar los ficheros **requirements** de cada proyecto y parsear esa información, pero dicho fichero no está presente en una gran cantidad de proyectos, por lo que se ha optado por un camino un poco más **radical**.

Usando la información cargada en el paso anterior, fundamentalmente la propiedad **git_url**, vamos a clonar cada proyecto en local para posteriormente procesarlos.

**NOTAS**: 
* El directorio ROOT_PATH debe existir antes de lanzar el siguiente script
* Nos aseguramos que el directorio no existe, porque este código, dado que puede tardar bastante, lo podemos ejecutar en diferentes momentos, es decir llegado un punto se podría abortar el proceso y lanzarlo más adelante sin que se resienta por ello.

In [9]:
os.chdir(ROOT_PATH)

for project in repository_projects.find({'pipeline_status':'INITIAL'}):
    print("Cloning project", project['name'], "...")
    path = ROOT_PATH + "/" + str(project["id"])
    if not os.path.isdir(path):
        os.system(CLONE_COMMAND.format(project["git_url"], project["id"]))
    project['pipeline_status'] = 'CLONED'
    repository_projects.update({'_id': project['_id']}, {"$set": project}, upsert=False)

Cloning project defcon25-public ...
Cloning project MachineLearningAction ...
Cloning project isf ...
Cloning project Deep-Image-Matting ...
Cloning project TWindbg ...
Cloning project visimportance ...
Cloning project vulcan ...
Cloning project WAF_Bypass_Helper ...
Cloning project programmable-agents_tensorflow ...
Cloning project keras-transform ...
Cloning project django_rest_example ...
Cloning project SkySpyWatch ...
Cloning project pypaperbak ...
Cloning project QQ_zone ...
Cloning project Imports-in-Python ...
Cloning project kinetics-i3d ...
Cloning project OSINT-SPY ...
Cloning project cifar-10-cnn ...
Cloning project neural_factorization_machine ...
Cloning project webcrawler ...
Cloning project solving-minesweeper-by-tensorflow ...
Cloning project parseNTFS ...
Cloning project attentional_factorization_machine ...
Cloning project ChatGirl ...
Cloning project Semantic_Segmentation ...
Cloning project captcha-svm ...
Cloning project pytorch-fitmodule ...
Cloning project trafa

En este punto tendríamos clonados todos los repositorios con los que construiremos el sistema de recomendación.

**NOTA**: La cantidad de datos descargados supera los 1.3Tb

# Procesamiento de los proyectos clonados

En esta fase, partiendo de los ficheros ya clonados en local, se identifican los ficheros README y se almacenan en la BBDD, por otra parte, se recorren línea por línea los ficheros Python y, usando expresiones regulares, se extraen las librerías que se están usando para almacenarlas también en la BBDD.

Comenzamos definiendo una función que dado un fichero python (extensión **.py**), lo recorre linea por linea y, usando expresiones regulares, buscamos todas las librerias que se estén usando, de forma esquemática estas son las acciones que se llevan a cabo:

 - Contemplar la posibilidad de que lo devuelto por la expresión regular sea una lista de librerías (import lib1,lib2,lib), en cuyo caso descomponemos la lista en sus librerías individuales
 - Se excluyen las librerías que empiezan por "**.**" ya que se trata de librerías internas de los proyectos
 - Se contempla que la librería tenga un alias eliminándolo
 - Se queda solo con el paquete principal
 - Se limpian espacios en blanco

Almacenamos esta lista en la propiedad **library** del proyecto en cuestión, también almacenamos la lista de líneas que declaran los imports por si en el futuro necesitamos volver a procesarlas.

In [10]:
def process_python_file(project, file_path):
    def process_expression(item):
        def insert(lib):
            lib = lib.strip()
            if '.' in lib:
                # Solo nos quedamos con el paquete principal
                lib = lib.split('.')[0]
            if not lib in library:
                library.append(lib)

        if not item.startswith('.'):
            if ',' in item:
                [insert(lib) for lib in item.split(',')]
            elif " as " in item:
                insert(item.split(" as ")[0])
            else:
                insert(item)

    library = project['library']
    library_lines = project['library_lines']

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            match = re.search(LIBRARY_PATTERN, line)
            if match:
                library_lines.append(line)
                if match.group(1) != None:
                    process_expression(match.group(1))
                else:
                    process_expression(match.group(2))

    project['library'] = library
    project['library_lines'] = library_lines

Por otra parte, definimos también una función que dado un fichero **README** lo lea y almacena en la propiedad **readme_txt**

In [11]:
def process_readme_file(project, file_path):
    with open(file_path, 'r') as f:
        project['readme_txt'] = f.read()

Finalmente recorremos los repositorios no procesados, como comentabamos anteriormente este proceso permite lanzarlo reiteradas veces, para cada repositorio analizamos su fichero **README** y cada uno de los ficheros Python para extraer las librerías.

In [12]:
for project in repository_projects.find({'pipeline_status':'CLONED'}):
    try:
        path = ROOT_PATH + "/" + str(project["id"])
        if os.path.isdir(path):
            print("Processing project", project["name"])
            for root, dirs, files in os.walk(path):
                for file in files:
                    try:
                        if file.endswith(FILE_LANGUAGE_EXTENSION):
                            process_python_file(project, os.path.join(root, file))
                        else:
                            if file.lower().startswith("readme."):
                                process_readme_file(project, os.path.join(root, file))
                    except:
                        pass
            project['pipeline_status'] = 'PROCESSED'
            repository_projects.update({'_id': project['_id']}, {"$set": project}, upsert=False) 
    except:
        print("Error procesing project {0} [{1}] - {2}".format(project['id'], project['name'], sys.exc_info()[0]))

Processing project defcon25-public
Processing project MachineLearningAction
Processing project isf
Processing project Deep-Image-Matting
Processing project TWindbg
Processing project visimportance
Processing project vulcan
Processing project WAF_Bypass_Helper
Processing project programmable-agents_tensorflow
Processing project keras-transform
Processing project django_rest_example
Processing project SkySpyWatch
Processing project pypaperbak
Processing project QQ_zone
Processing project Imports-in-Python
Processing project kinetics-i3d
Processing project OSINT-SPY
Processing project cifar-10-cnn
Processing project neural_factorization_machine
Processing project webcrawler
Processing project solving-minesweeper-by-tensorflow
Processing project parseNTFS
Processing project attentional_factorization_machine
Processing project ChatGirl
Processing project Semantic_Segmentation
Processing project captcha-svm
Processing project pytorch-fitmodule
Processing project trafaret
Processing project p

## Detección del idioma del proyecto

Dado que uno de los pilares del sistema de detección es el Procesamiento del Lenguaje Natural, uno de los primeros pasos es detectar el idioma en el que está redactado el fichero **README** fuente de nuestros textos. Dado que la mayoría de proyectos están redactados en Ingles, y en aras de simplificar el modelo, vamos a eliminar todos los proyectos que no estén descritos en este idioma.

Para ello vamos a usar la función **detect** de la librería **langdetect**, implementaremos un *wrapper* sobre ella que tendrá en cuenta los posibles errores así como la posibildad de que el fichero README esté vacío.

In [13]:
from langdetect import detect

def detect_language(project):
    language = None
    try:
        if len(project['readme_txt']):
            language = detect(project['readme_txt'])
    except:
        pass
    return (language)

Obtenemos los proyectos que, por un motivo un otro, no estén en **Inglés**

In [14]:
projects_not_english = [project for project in list(repository_projects.find()) if detect_language(project) != 'en']

Comprobamos que el número de proyectos que vamos a borrar no es muy elevado con respecto al total.

In [15]:
print("English proyects number  :", len(projects_not_english))
print("Total number of proyects :", repository_projects.count() )

English proyects number  : 9
Total number of proyects : 36


Finalmente borramos los proyectos que no nos interesan

In [16]:
for project in projects_not_english:
    repository_projects.delete_one({'id': project["id"]})

Mostramos el número de proyectos que han quedado

In [17]:
print("Total number of proyects :", repository_projects.count() )

Total number of proyects : 27


# Procesamiento de los datos

Una vez tenemos la base de los metadatos que vamos a usar, el siguiente paso es procesarlos de cara a tenerlos preparados para su uso directo por parte de los modelos.

## Proyectos

En cuanto a lo que a los proyectos se refiere, procesamos las descripciones quedándonos con las palabras diferentes que encontramos en cada una de ellas, eliminamos puntuaciones, números y palabras irrelevantes como nombres personales y palabras comunes que no aportan significado (denominados "stop words"), esto evitará que se formen tópicos en torno a ellos

Además, utilizamos el stemmer Snowball, también llamado Porter2 stemmer, para detectar palabras similares presentes en diferentes formatos (eliminar sufijo, prefijo, etc.). Snowball es un lenguaje desarrollado por M.F. Porter, para definir de forma eficiente stemmers. Este algoritmo de derivación es el más utilizado en el dominio del procesamiento del lenguaje natural.

Para hacer el procesamiento usaremos la librería **nltk** (Natural Language Toolkit), proporciona un gran número de métodos que cubren diferentes temas en el dominio de los datos del lenguaje humano, como la clasificación, derivación, etiquetado, análisis y razonamiento semántico.

Creamos una función que dado un texto lo descompone en las palabras con significado que lo componen. Vamos a excluir los siguientes tipos de palabras:

- Nombres propios
- Palabras consideradas como "Stop Words"
- Números

Adicionalmente aplicamos el proceso de "steamming" que comentabamos anteriormente

In [18]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [19]:
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

In [20]:
def get_words(text):
    def add_word(word):
        word = word.lower()
        if word not in stop_words and not word.replace('.', '').isdigit():
            words.append(stemmer.stem(word))

    words = []
    for chunk in nltk.ne_chunk(nltk.pos_tag(tokenizer.tokenize(text))):
        # nltk.word_tokenize    devuelve la lista de palabras que forman la frase (tokenización)
        # nltk.pos_tag          devuelve el part of speech (categoría) correspondiente a la palabra introducida
        # nltk.ne_chunk         devuelve la etiqueta correspondiente al part of speech (POC)
        try:
            if chunk.label() == 'PERSON':
                # PERSON es un POC asociado a los nombres propios, los cuales no vamos a añadir
                pass
            else:
                for c in chunk.leaves():
                    add_word(c[0])
        except AttributeError:
            add_word(chunk[0])

    return words


## Librerías

En cuanto a las librerías solo vamos tener en cuenta las librerías de "**terceros**", es decir vamos a excluir las librerías definidas en los propios proyectos y las librerías del sistema, las primeras porque obviamente no las van a tener otros proyectos y lo único que haría sería incluir ruido y las segundas, las del sistema, por ser demasiado comunes.

Para poder catalogar las librerías de terceros, vamos a usar como referencia las librerías disponibles en [PyPi](https://pypi.python.org/pypi). Realizaremos un proceso de scraping sobre su web, almacenaremos las librerías un una colección de nuestra BBDD Mongo.

**NOTA**: Vamos a sustituir el caracter "**-**" por "**_**", esto se hace porque nos encontraremos casos en los que la librería se llame algo como **some-word** y la instalación se deba hacer con ese nombre **pip install some-word**, sin embargo, cuando la encontremos en el código pasará a llamarse **some_word**, es decir encontraremos algo como **import some_word**

In [24]:
import requests

PYPI_LIST_URL = 'https://pypi.python.org/simple/'

def get_repository_library():
    client = MongoClient('localhost', 27017)
    db = client.github_t5
    return db.library

def insert_library(library_name):
    library_name = library_name.lower().replace("-", "_")
    if repository_library.find_one({"name": library_name}):
        print("Library {0} is already included in the repository".format(library_name))
    else:
        library = {
            'name': library_name
        }
        repository_library.insert(library)

repository_library = get_repository_library()

In [ ]:
content = requests.get(PYPI_LIST_URL)

for line in content.text.split('\n'):
    match = re.search("'>([\w\-\.]+)", line)
    if match:
        insert_library(match.group(1))

Vamos a definir una función que filtre las librerías que no pertenezcan a la colección creada en el paso anterior

In [25]:
def process_library(library):
    library_processed = []
    for lib in library:
        if repository_library.find_one({"name": lib}):
            library_processed.append(lib)
    return library_processed

A continuación recorremos los proyectos aplicándole tanto el procesamiento a los ficheros **README** como a las **librerías**.

In [26]:
for project in repository_projects.find({'pipeline_status':'PROCESSED'}):
    try:
        print("Processing",project["name"])
        project['readme_words'] = get_words(project['readme_txt'])
        project['library'] = process_library(project['library'])
        project['pipeline_status'] = 'DONE'
    
        repository_projects.update({'_id': project['_id']}, {"$set": project}, upsert=False)
    except:
        print("Error procesing project {0} [{1}] - {2}".format(project['id'], project['name'], sys.exc_info()[0]))
        pass

Processing defcon25-public
Processing isf
Processing Deep-Image-Matting
Processing TWindbg
Processing visimportance
Processing vulcan
Processing WAF_Bypass_Helper
Processing programmable-agents_tensorflow
Processing keras-transform
Processing django_rest_example
Processing SkySpyWatch
Processing pypaperbak
Processing Imports-in-Python
Processing kinetics-i3d
Processing OSINT-SPY
Processing cifar-10-cnn
Processing neural_factorization_machine
Processing attentional_factorization_machine
Processing ChatGirl
Processing Semantic_Segmentation
Processing pytorch-fitmodule
Processing trafaret
Processing privacy-protocols
Processing lp-diet
Processing repotoddy
Processing deeppavlov
Processing rfcat-firsttry


Finalmente eliminaremos de nuestro repositorio los proyectos que no tienen librerías o que no tienen ningun elemento en su lista de palabras (la descomposición de los ficheros **readme**)

In [27]:
for project in repository_projects.find({'pipeline_status':'DONE'}):
    if len(project['library']) == 0 or len(project['readme_words']) == 0:
        print("Deleting",project["name"])
        repository_projects.delete_one({'id': project["id"]})

.

<span style="color: #12297c; font-family: Arial; font-size: 3em;">Implementación de modelos</span>

Tras la fase anterior, tendríamos una colección MongoDB con cada uno de los proyectos de nuestro "pre-corpus" con la lista de librerías que cada proyecto usa, así como su descripción **"extendida"** extraída de su fichero **README**. Estaríamos en disposición por tanto de comenzar la implementación de nuestro recomendador.

Vamos a implementar dos versiones del recomendador, una que buscará similitudes basándose en las descripciones de los proyectos y otra que se basará en las librerías que cada proyecto usa para determinar proyectos similares.
Para la primera de las opciones se usará un modelo **LSA** (Latent Semantic Analysis) usando procesamiento de lenguaje natural, para la segunda se usará un simple **"Count Vectorizer"**, dado que la aproximación anterior no es adecuada para documentos con pocos términos, en nuestro caso un documento sería el listado de las librerías usadas.


## Carga de los datos


Comenzamos importando las librerías que vamos a usar en esta fase del proyecto.

In [28]:
import os
import uuid
import re
import nltk
import json
import itertools
import gensim
import random
import numpy as np
import matplotlib.pyplot as plt
from pymongo import MongoClient
from collections import defaultdict
from collections import OrderedDict
from gensim import corpora, models, similarities, matutils
from gensim.models import CoherenceModel

Slow version of gensim.models.doc2vec is being used


Definimos una funcion para cargar los datos de los proyectos, entre los datos a cargar está el fichero **readme** y las librerías usadas, datos que serán los que usemos para calcular las similitudes entre los proyectos.

In [29]:
def load_projects(max_projects=50000):
    def get_repository():
        client = MongoClient('localhost', 27017)
        db = client.github_t5
        return db.projects
    projects_repository = get_repository()
    return list(projects_repository.find({'pipeline_status':'DONE'}).limit(max_projects))

Usando la funcíon anterior cargamos los datos, dado que el proceso es bastante pesado solo cargaremos un subconjunto de las mismas

In [30]:
projects = load_projects()

A modo de ejemplo, mostramos las primeras 5 librerías de uno de los proyectos

In [31]:
projects[0]['library'][:5]

['parser', 'pyparsing', 're', 'idc', 'ctypes']

Creamos una función que construye una lista con todas las descripciones de todos los proyectos

In [32]:
def get_texts_from_readme(projects):
    texts = []
    [texts.append(project['readme_words']) for project in projects]
    return texts

Ejecutamos la función que acabamos de definir, tendremos una lista de listas, en la que para cada proyecto tendremos las palabras que lo definen.

In [33]:
texts_from_readme = get_texts_from_readme(projects)

A modo de ejemplo, mostramos las 5 primeras entradas de uno de los proyectos.

In [34]:
texts_from_readme[0][:5]

['make', 'build', 'dynam', 'librari', 'window']

Y una parte del contenido del fichero README

In [35]:
projects[0]['readme_txt'][:2000]

"## Usage\n\nRun `make` to build the dynamic library, Windows DLLs, and wrapper for Python. The _Makefile_ uses MinGW to cross compile the Windows DLLs, so you will get errors if you don't have it installed. If you are on macOS, it is sufficient to `brew install mingw-w64`.\n\nYou can do a basic test of the disassembler by running _test.py_. It will attempt to disassemble every 4 byte sequency in _test.bin_.\n\nIf you want use the IDA plugin, you will need to copy the files to the appropriate directories. Assuming that *IDA_PATH* is your IDA directory, copy the following files:\n\n  - riscv_32.dll, riscv_64.dll -> *IDA_PATH/*\n  - pyriscv.py -> *IDA_PATH/python/*\n  - ida-plugin/riscv.py -> *IDA_PATH/procs/*\n\nThe C library exposes a simple interface:\n\n```\nEXPORT void disassemble(inst_t *inst, uint32_t pc, const uint8_t *buf);\nEXPORT const char *mnemonics[];\nEXPORT const char *registers[];\nEXPORT const unsigned int num_registers;\n```\n\nThe Python wrapper exposes a similar inte

# Modelo LSA/LSI: Latent Semantic Analysis / Latent Semantic Indexing

## Principios teóricos

Para identificar la similitud entre los proyectos basándonos en su descripción, entendiéndose como tal su fichero README, utilizamos el "análisis semántico latente" (LSA, usando la abreviatura en inglés), que es una técnica ampliamente utilizada en el procesamiento del lenguaje natural. LSA transforma cada texto en un vector, en un espacio de características. En nuestro caso, las características son palabras que ocurren en las descripciones. A continuación, se crea una matriz que contiene todos los vectores: las columnas representan las descripciones de los proyectos y las filas representan palabras únicas. Por consiguiente, el número de filas puede ascender a decenas de miles de palabras. 

Con el fin de identificar las características relevantes de esta matriz, usaremos la "descomposición de valores singulares" (SVD, usando la abreviatura en inglés), que es una técnica de reducción de dimensión, se utiliza para reducir el número de líneas -palabras-, manteniendo y resaltando la similitud entre columnas-descripción -. La dimensión de esta matriz de aproximación se establece mediante un hiperparámetro que es el número de temas, comúnmente llamado como tópicos. En este marco, un tópico consiste en un conjunto de palabras con pesos asociados que definen la contribución de cada palabra a la dirección de este tópico. Basándose en esta matriz de aproximación de baja dimensión, la similitud entre dos columnas -descripciones- se calcula utilizando el coseno del ángulo entre estos dos vectores.


**NOTA**: Generalmente LSA y LSI se utilizn indistintamente para referirse al mismo concepto.

## Creación del diccionario
 
El diccionario está formado por la concatenación de todas las palabras que aparecen en algún resumen de alguno de los proyectos. Básicamente esta función mapea cada palabra única con su identificador. Es decir, si tenemos N palabras, lo que conseguiremos al final es que cada proyecto sea representada mediante un vector en un espacio de N dimensiones.
 
Para ello, partiendo de la lista creada en el paso anterior, usaremos la función **corpora** del paquete **gensim**.

El diccionario consiste en una concatenación de palabras únicas de todas las descripciones. Gensim es una biblioteca eficiente para analizar la similitud semántica latente entre documentos.
Este módulo implementa el concepto de Diccionario - un mapeo entre palabras y
sus entes ids.

Los diccionarios pueden ser creados a partir de un corpus y luego pueden ver las frecuencia del documento (eliminación de palabras comunes mediante el método func: `Dictionary.filter_extremes`), guardado / cargado desde el disco (vía: func: `Dictionary.save` y: func:` Dictionary.load`), fusionado con otro diccionario (: func: `Dictionary.merge_with`) etc.

In [36]:
dictionary = corpora.Dictionary(texts_from_readme)
dictionary

Podemos ver la longitud del diccionario creado

In [37]:
print(dictionary)

Dictionary(1883 unique tokens: ['isweekend', 'protocol', 'accord', 'question', 'directori']...)


La función **token2i** asigna palabras únicas con sus ids. En nuestro caso, la longitud del diccionario es igual a *N* palabras lo que significa que cada descripción del proyecto será representada a través de un espacio vectorial de *N* dimensiones

Mostramos las primeras 10 entradas

In [38]:
list(itertools.islice(dictionary.token2id.items(), 0, 10))

[('isweekend', 1425),
 ('protocol', 193),
 ('accord', 1377),
 ('question', 1122),
 ('directori', 2),
 ('perform', 955),
 ('convnet', 1126),
 ('26817e', 1128),
 ('us', 282),
 ('run_exampl', 1567)]

## Creación del Corpus

Crearemos un corpus con la colección de todos los resúmenes previamente pre-procesados y transformados usando el diccionario. Vamos a convertir los textos a un formato que gensim puede utilizar, esto es, una representación como bolsa de palabras (BoW). Gensim espera ser alimentado con una estructura de datos de corpus, básicamente una lista de "sparce vectors", estos constan de pares (id, score), donde el id es un ID numérico que se asigna al término a través de un diccionario. 

Mostramos el numero de elementos únicos que existen en los documentos (los que componen el diccionario) y el numero de textos (correspondientes a las descripciones de los proyectos).

In [39]:
print('Number of unique tokens : %d' % len(dictionary))
print('Number of documents     : %d' % len(texts_from_readme))

Number of unique tokens : 1883
Number of documents     : 27


In [40]:
def create_corpus(dictionary, texts):
    return [dictionary.doc2bow(text) for text in texts]

In [41]:
corpus = create_corpus(dictionary, texts_from_readme)

A modo de ejemplo, mostramos las 5 primeras entradas del primer proyecto

In [42]:
corpus[0][:5]

[(0, 2), (1, 3), (2, 2), (3, 3), (4, 1)]

## Creación del modelo TFID

Un alto peso en tf-idf se alcanza por una alta frecuencia en un Documento y una baja frecuencia en toda la colección de documentos; los pesos tienden a filtrar términos comunes. Para la creación de este corpus, vamos a usar la función **TfidfModel** del objeto **models** (perteneciente a la librería *gemsim*).


In [43]:
def create_tfidf(corpus):
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    return corpus_tfidf

In [44]:
corpus_tfidf = create_tfidf(corpus)

En corpus tenemos, para cada project, una lista con sus palabras y el tfidf de cada una. Mostramos las 10 primeras entradas del <primer elemento

In [45]:
corpus_tfidf[0][:10]

[(0, 0.028236579108337465),
 (1, 0.07329182662576354),
 (2, 0.028236579108337465),
 (3, 0.09281087388977204),
 (4, 0.03093695796325735),
 (5, 0.016513052537392683),
 (6, 0.020624638642171563),
 (7, 0.04886121775050903),
 (8, 0.03093695796325735),
 (9, 0.024430608875254514)]

Si queremos saber que palabra es cada uno de estos términos podemos consultar el diccionario

In [46]:
print(dictionary[8],",",dictionary[19],",", dictionary[88])

w64 , r , ad


## Determinación del número de topicos

Con los elementos que tenemos hasta ahora ya estamos listos para entrenar el modelo LSI/LDA. Pero antes que nada, como dicen los anglosajones "*el elefante en la habitación*", **¿Cuál es el número correcto de tópicos?**

LSA busca identificar un conjunto de tópicos relacionados las descripciones de los proyectos. El número de estos temas N es igual a la dimensión de la matriz de aproximación resultante de la técnica de reducción de dimensión SVD. Este número es un hiperparámetro que se debe ajustar cuidadosamente, es el resultado de la selección de los N valores singulares más grandes de la matriz del corpus tf-idf. 

En principio no hay una respuesta correcta o fácil para la pregunta inicial, depende tanto de los datos que tengamos como de la aplicación que queramos dar a estos datos. Por ejemplo, en ciertos casos puede ser útil tener un número pequeño de tópicos, de mara que se puedan interpretar y "etiquetar".



Vamos a crear una serie de modelos variando el número de tópicos y posteriormente los evaluaremos buscando el número óptimo de tópicos.


**NOTA**: Gensim puede provocar cuelgues por un uso excesivo de memoria, así que hay que tener en cuenta esta limitación cuando se indica por ejemplo el número de tópicos. Antes de entrenar su modelo, se puede obtener una estimación aproximada del uso de la memoria usando la siguiente fórmula:

</br>

<center>**8 bytes** *x* **numero_terminos** *x* **numero_topicos** *x* **3**</center>

Así pues, dadas las características de máquina con la que se realizán los cálculos vamos a indicar **120** tópicos como máximo

In [47]:
trained_models = OrderedDict()
for num_topics in range(10, 121, 10):
    print("Training LDA(k=%d)" % num_topics)
    lda = models.LdaMulticore(
        corpus_tfidf, id2word=dictionary, num_topics=num_topics, workers=4,
        passes=10, iterations=100, random_state=42, 
        eval_every=None,      # No evalúa la perplejidad del modelo, lleva demasiado tiempo.
        alpha='asymmetric',   # Demuestra ser mejor que simétrico en la mayoría de los casos
        decay=0.5, offset=64  # Mejores varlores para Hoffman paper
    )
    trained_models[num_topics] = lda

Training LDA(k=10)
Training LDA(k=20)
Training LDA(k=30)
Training LDA(k=40)
Training LDA(k=50)
Training LDA(k=60)
Training LDA(k=70)
Training LDA(k=80)
Training LDA(k=90)
Training LDA(k=100)
Training LDA(k=110)
Training LDA(k=120)


Podemos mostrar uno de los modelos creados

In [48]:
trained_models[10]

### Evaluación usando la *Coherencia*

Vamos a evaluar cada uno de nuestros modelos de LDA utilizando la coherencia. La **coherencia** es una medida de cuán interpretables son los topicos para los humanos. Se basa en la representación de temas como las N palabras más probables para un topico en particular. Más específicamente, dada la matriz de *topic-term* para LDA, clasificamos cada tema de pesos más altos a más bajos y luego seleccionamos los primeros N términos.

La coherencia esencialmente mide cuán similares son estas palabras entre sí. Existen varios métodos para hacerlo, la mayoría de los cuales han sido explorados en el documento ["Exploring the Space of Topic Coherence Measures"](https://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf). Los autores realizaron un análisis comparativo de varios métodos, correlacionándolos con juicios humanos. El método llamado coherencia "c_v" resultó ser el más altamente correlacionado. Este y varios de los otros métodos se han implementado en gensim.models.CoherenceModel. Usaremos esto para realizar nuestras evaluaciones.



Recorremos la lista de modelos y obtenemos la **coherencia** de cada uno de ellos, además llevaremos un registro para determinar cual de ellos obtiene el mejor resultado.

In [49]:
best_coherence = 0
best_coherence_key = 0
for key, value in trained_models.items():
    cm = CoherenceModel(model=value, texts=texts_from_readme, dictionary=dictionary, coherence='c_v')
    actual_coherence = cm.get_coherence()
    print("Topic number: {} - Coherence: {}".format(key,actual_coherence))
    if actual_coherence > best_coherence:
        best_coherence = actual_coherence
        best_coherence_key = key
print("\nBest model has {} topics and a coherence of {}".format(best_coherence_key,best_coherence))

Topic number: 10 - Coherence: 0.6746561302507453
Topic number: 20 - Coherence: 0.6995934153834668
Topic number: 30 - Coherence: 0.7215333235188932
Topic number: 40 - Coherence: 0.72088870991336
Topic number: 50 - Coherence: 0.7273327122959137
Topic number: 60 - Coherence: 0.7272353970334672
Topic number: 70 - Coherence: 0.7278426205261573
Topic number: 80 - Coherence: 0.7363613347398256
Topic number: 90 - Coherence: 0.7294358751795041
Topic number: 100 - Coherence: 0.7446173646569327
Topic number: 110 - Coherence: 0.7391535053844792
Topic number: 120 - Coherence: 0.7369847301148149

Best model has 100 topics and a coherence of 0.7446173646569327


La prueba nos ha dado como resultado que con **20** tópicos obtenemos los mejores resultados.

Finalmente creamos constantes para los valores seleccionados.
 * **TOTAL_LSA_TOPICS**: Limita el numero de terminos, por supuesto tiene que ver con el tamaño de la muestra, mientras más proyectos tengamos mas terminos tendremos y por tanto la reduccion seria mayor, estamos clusterizando las proyectos en TOTAL_TOPICOS_LSA clusters
 
* **SIMILARITY_THRESHOLD** Umbral de similitud que se debe superar para que dos proyectos se consideren similares (en cualquier caso, posteriormente usaremos los proyectos ordenados por similitud

In [50]:
best_coherence_key = 100
TOTAL_LSA_TOPICS = best_coherence_key
SIMILARITY_THRESHOLD = 0.4

In [51]:
TOTAL_LSA_TOPICS

100

## Definición del modelo LSA

### Funciones auxiliares

Definimos una serie de funciones auxiliares para ayudarnos en las siguientes operaciones.

**get_similarities_by_description**

Dado un documento (correspondiente a la descripción de un proyecto), nos determina la lista de proyectos que superan el umbral de similitud. Para cada proyecto que supere el umbral, almacenaremos el índice dentro de la matriz de proyectos, para localizarla posteriormente, y el grado de similitud. 

In [52]:
def get_similarities_by_description(model, dictionary, similarity_matrix, doc):
    ''' Calcula las similitudes de un documento, expresado este como una lista de palabras'''
    project_similarities = defaultdict(float)

    # Convertimos el documento al espacio LSI
    vec_bow = dictionary.doc2bow(doc)
    vec_lsi = model[vec_bow]

    similarities = similarity_matrix[vec_lsi]
    similarities = sorted(enumerate(similarities), key=lambda item: -item[1])

    for sim in similarities:
        similarity_project = int(sim[0])
        similarity_score = sim[1]
        if similarity_score > SIMILARITY_THRESHOLD:
            project_similarities[similarity_project] = similarity_score

    return (project_similarities)

**show_project_similarities**

Dada una lista de proyectos similares a uno dado, lo imprime por pantalla.

In [53]:
def show_project_similarities(projects, project_similarities):
    for project_id in sorted(project_similarities, key=project_similarities.get, reverse=True)[:10]:
         print("Project: {0}: - Similarity: {1}".format(projects[project_id]["name"], project_similarities[project_id]))

**get_library_similarities**

Dada una lista de proyectos similares a uno dado y una lista de librerías, retorna las librerías que usan esos proyectos y que no están en la lista pasada por parámetro. Además para cada librería calcula su índice de similitud, basándose en la similitud de los proyectos a los que pertenece. 

In [54]:
def get_library_similarities(projects, project_similarities, project_libraries):
    library_similarities=defaultdict(float)
    for project_id in sorted(project_similarities, key=project_similarities.get, reverse=True)[:10]:
        similarity = project_similarities[project_id]
        libraries = projects[project_id]["library"]
        for library in libraries:
            if library not in project_libraries:
                library_similarities[library] += similarity
    return library_similarities

**show_library_similarities**

Dada una lista de proyectos similares a uno dado y una lista de librerías, retorna las librerías que usan esos proyectos y que no están en la lista pasada por parámetro. Además apra cada librería calcula su índice de similitud, basándose en la similitud de los proyectos a los que pertenece. 

In [55]:
def show_library_similarities(library_similarities):
    for library in sorted(library_similarities, key=library_similarities.get, reverse=True)[:10]:
        print("Library: {0}: - Score: {1}".format(library, library_similarities[library]))

### Creación del modelo

Para ello usaremos la librería **models** que nos ofrece gensim

In [56]:
model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=TOTAL_LSA_TOPICS)

Podemos ver como influyen las palabras en la determinación de los diferentes tópicos, por ejemplo mostramos los 3 primeros tópicos

In [57]:
for i, topic in enumerate(model.print_topics(3)):
    print('Topic {}:'.format(i))
    print(topic[1].replace(' + ', '\n'))
    print('')

Topic 0:
-0.251*"train"
-0.194*"model"
-0.141*"data"
-0.113*"br"
-0.108*"kinet"
-0.107*"rgb"
-0.103*"dataset"
-0.102*"ml"
-0.099*"trafaret"
-0.097*"repotoddi"

Topic 1:
-0.245*"train"
-0.204*"vgg"
0.203*"trafaret"
-0.168*"pre"
-0.148*"weight"
0.136*"repotoddi"
-0.120*"br"
0.115*"import"
-0.107*"model"
0.106*"osint"

Topic 2:
-0.231*"val_accuraci"
-0.231*"val_loss"
-0.204*"epoch"
-0.168*"accuraci"
-0.168*"import"
-0.149*"modul"
-0.144*"self"
0.142*"osint"
-0.137*"trafaret"
-0.128*"loss"



Usando nuestro modelo LSI construimos la matriz de similitud

In [58]:
similarity_matrix = gensim.similarities.MatrixSimilarity(model[corpus_tfidf])

### Prueba de concepto

A modo de ejemplo, como prueba de concepto, vamos a determinar los proyectos similares a uno dado. En primer lugar definimos las propiedades de nuestro proyecto **fake**.

In [59]:
poc_library = [ "logging", "time", "printer", "distutils", "scapy", "requests", "bs4", "pysnmp", "paramiko", "nmap", "unittest" ]
poc_readme_words = get_words("works on Windows and write down some installation instructions")

Calculamos la lista de proyectos similares

In [60]:
project_similarities_by_description = get_similarities_by_description(model, dictionary, similarity_matrix, poc_readme_words)

Mostramos los 10 proyectos más similares

In [61]:
show_project_similarities(projects, project_similarities_by_description)

Project: defcon25-public: - Similarity: 0.7655923366546631
Project: deeppavlov: - Similarity: 0.4258296489715576


El objetivo final no no es encontrar proyectos similares, sino encontrar las librerias que usan esos proyectos similares. Así pues, recorremos esos proyectos e identificamos esas librerías, descartando las que nuestro proyecto ya incluye.

Recorremos los 20 proyectos más similares y calculamos el scoring de cada librería usada en esos proyectos

In [62]:
library_similarities_by_description = get_library_similarities(projects, project_similarities_by_description, poc_library)

Mostramos las 10 librerías más usadas por los proyectos similares

In [63]:
show_library_similarities(library_similarities_by_description)

Library: re: - Score: 1.1914219856262207
Library: parser: - Score: 0.7655923366546631
Library: ctypes: - Score: 0.7655923366546631
Library: pyparsing: - Score: 0.7655923366546631
Library: idc: - Score: 0.7655923366546631
Library: build_utils: - Score: 0.4258296489715576
Library: uuid: - Score: 0.4258296489715576
Library: tqdm: - Score: 0.4258296489715576
Library: fasttext: - Score: 0.4258296489715576
Library: nltk: - Score: 0.4258296489715576


# Modelo CountVectorizer

Otro acercamiento que podemos realizar es usar como nivel de similitud las propias librerías que un proyecto usa, es decir buscaremos proyectos que usen las mismas librerías que nosotros ya estamos usando y mostraremos otras librerias que esos mismos proyectos estén usando y nuestro proyecto no.

En este caso los documentos serán muy pequeños, así que técnicas como LSI/LDA no son apropidas dado que están basados en estadísticas y con documentos pequeños no tendríamos suficiente información, se podrían usar técnicas como
[Word network topic model](https://dl.acm.org/citation.cfm?id=2974795), pero en este caso vamos a usar una aproximacion más simple y usaremos un modelo **CountVectorizer** de extracción de características.

In [121]:
import scipy as sp
from sklearn.feature_extraction.text import CountVectorizer

El modelo que construiremos necesita como entrada textos, así que transformamos nuestras listas de palabras en textos, en los que las librerías estarán separados por espacios.

In [122]:
global_texts = [" ".join(project['library']) for project in projects]

In [123]:
global_texts[:2]

['tensorflow numpy hello nltk',
 'modules argparse config requests clearbit shodan re steganography bitcoin email']

Usaremos un objeto de **CountVectorizer**, el cual convierte una colección de documentos de texto en una matriz de conteos de tokens. 

Useremos el parámetro **min_df**, denomina comunmente como **corte**, con el valor **2**; esto hará que al construir el vocabulario, se ignoren los términos que tienen una frecuencia de documento más baja que el umbral dado

In [124]:
model_countVectorizer = CountVectorizer(min_df=2)

Usando el objeto creado, realizamos la transformación de nuestros datos

In [125]:
data = model_countVectorizer.fit_transform(global_texts)

A continuación definimos una función que usando los datos anteriores, el objeto countVectorizer y una lista de librerias, calcula la distancia coseno con cada una de las listas de librerias de los proyectos objeto de estudio. La **distancia coseno**  no es propiamente una distancia sino una medida de similaridad entre dos vectores en un espacio que tiene definido un producto interior. 

In [126]:
def get_similarities_by_library(train_data, doc, model):
    def cos_distance(v1, v2):
        return 1 - (v1 * v2.transpose()).sum() / (sp.linalg.norm(v1.toarray()) * sp.linalg.norm(v2.toarray()))
    
    new_doc_vectorized = model.transform([doc])[0]
    library_similarities = defaultdict(float)
    for i, doc_vec in enumerate(train_data):
        library_similarities[i] = 1 - cos_distance(doc_vec, new_doc_vectorized)
    return library_similarities

Vamos a realizar una prueba de concepto usando la lista de librería que ya tenemos creada, primero la convertimos en un documento tal como se ha comentado anteriormente.

In [127]:
doc_from_library = " ".join(poc_library)
doc_from_library

'logging time printer distutils scapy requests bs4 pysnmp paramiko nmap unittest'

Obtenemos los proyectos más similares en función de las librerías que comparten

In [128]:
project_similarities_by_library = get_similarities_by_library(data, doc_from_library, model_countVectorizer)

Como hicimos con el análisis por descripciones, podemos mostrar los proyectos ordenados por similitud

In [129]:
show_project_similarities(projects, project_similarities_by_library)

Project: Imports-in-Python: - Similarity: nan
Project: isf: - Similarity: 0.7071067811865475
Project: SkySpyWatch: - Similarity: 0.4743416490252569
Project: vulcan: - Similarity: 0.44721359549995787
Project: rfcat-firsttry: - Similarity: 0.3651483716701107
Project: deeppavlov: - Similarity: 0.34641016151377535
Project: trafaret: - Similarity: 0.2981423969999719
Project: OSINT-SPY: - Similarity: 0.2581988897471611
Project: privacy-protocols: - Similarity: 0.2581988897471611
Project: django_rest_example: - Similarity: 0.2581988897471611


Aunque como ya dijimos entonces, el objetivo es recomendar librerías, no proyectos. Así que obtenemos la lista a partir de nuestra lista de proyectos

In [130]:
library_similarities_by_libraries = get_library_similarities(projects, project_similarities_by_library, poc_library)

Finalmente mostramos el resultado

In [131]:
show_library_similarities(library_similarities_by_libraries)

Library: machines: - Score: nan
Library: re: - Score: 1.9750066011175664
Library: datetime: - Score: 1.3770930972861652
Library: setuptools: - Score: 1.3499650437608661
Library: csv: - Score: 1.2679654060389902
Library: select: - Score: 1.0722551528566582
Library: functools: - Score: 1.0052491781865194
Library: numbers: - Score: 1.0052491781865194
Library: argparse: - Score: 0.9653056709337086
Library: multiprocessing: - Score: 0.8207518105390322


.

<span style="color: #12297c; font-family: Arial; font-size: 3em;">Rendimiento del modelo</span>

Divideremos el conjunto de datos en dos subconjuntos, uno de **entrenamiento** con el **70%** de los datos y otro de **test** con el **30%**. Mostramos el numero total de proyectos para verificar que se distribuyen correctamente los dos subconjuntos

In [77]:
num_projects = len(projects)
print("Total data set size:",num_projects)

Total data set size: 27


Antes de hacer la partición, los desordenamos, de otra forma los subconjuntos estarían sesgados dado que están ordenados por el **id** el cual depende de la fecha en la que se creó.

In [78]:
random.shuffle(projects)

### Conjunto de entrenamiento 

Nos quedamos con el 70% de los datos

In [79]:
projects_train = projects[:int(0.7*num_projects)]
print("Training data set size:",len(projects_train))

Training data set size: 18


### Conjunto de tests 
Nos quedamos con el 30% restante

In [80]:
projects_test = projects[int(0.7*num_projects):]
print("Test data set size:",len(projects_test))

Test data set size: 9


# Creación del modelo

Obtenemos la lista con las descripciones de los proyectos del conjunto de entrenamiento

In [81]:
texts_train = get_texts_from_readme(projects_train)
print(texts_train[1])

['cactuscon', 'http', 'static1', 'squarespac', 'com', 'static', '56b38e2aa3360ce530d55c24', '56b3fc1bf850820fe762454d', 'format', '1500w', 'http', 'cactuscon', 'com', 'us', 'major', 'updat', 'come', 'blackhat', 'arsenal', 'europ', 'stay', 'tune', 'someth', 'secret', 'osint', 'spi', 'search', 'use', 'osint', 'open', 'sourc', 'intellig', 'perform', 'osint', 'scan', 'email', 'domain', 'ip_address', 'organ', 'use', 'osint', 'spi', 'use', 'cyber', 'crime', 'investig', 'order', 'find', 'deep', 'inform', 'target', 'osint', 'spi', 'document', 'beta', 'readm', 'author', 'sk_secur', 'version', 'websit', 'osint', 'spi', 'com', 'overview', 'tool', 'scan', 'ip', 'address', 'domain', 'email', 'address', 'btc', 'bitcoin', 'address', 'devic', 'find', 'latest', 'bitcoin', 'block', 'inform', 'list', 'cipher', 'support', 'particular', 'websit', 'server', 'whether', 'particular', 'websit', 'vulner', 'heartble', 'dump', 'contact', 'messag', 'skype', 'databas', 'malwar', 'malic', 'file', 'remot', 'licens', 

Creamos el diccionario

In [82]:
dictionary_train = corpora.Dictionary(texts_train)

In [83]:
print(dictionary_train)

Dictionary(1534 unique tokens: ['isweekend', 'protocol', 'accord', 'question', 'directori']...)


Por último creamos el corpus y el corpus tfidf a partir de él

In [84]:
corpus_train = create_corpus(dictionary_train, texts_train)

In [85]:
corpus_tfidf_train = create_tfidf(corpus_train)

Con los elementos creados entrenamos el modelo a evaluar

In [86]:
model_train = models.LsiModel(corpus_tfidf_train, id2word=dictionary_train, num_topics=TOTAL_LSA_TOPICS)

Creamos la matriz de similaritud

In [87]:
similarity_matrix_train = similarities.MatrixSimilarity(model_train[corpus_tfidf_train])

Definimos una funcion que calcule el acierto de una predicción, se basa en el porcentaje de librerias que se han predicho con respecto al total de librerías.

In [88]:
def score(actual, predicted):
    actual_len = len(actual)
    predictec_success = set(actual).intersection(predicted)
    predictec_success_len = len(predictec_success)
    return(predictec_success_len/actual_len)

Por otra parte definimos otra función que obtiene la predicción para un proyecto y, usando la función anterior, calcula el score de dicha predicción.

In [89]:
def predict_libraries_from_description(model, dictionary, similarity_matrix, project):  
    get_similarities_by_library(train_data, doc, model)
    library_similarities = get_library_similarities(projects, project_similarities, [])
    # Ordernar y pasar a lista, tomamos el mismo número de librerías que las que tiene el proyecto real
    num_lib = len(project['library'])
    project['library_prediction'] = sorted(library_similarities, key=library_similarities.get, reverse=True)[:num_lib]
    project['score'] = score(project['library'], project['library_prediction'])

Finalmente obtenemos la predicción para todos los proyectos del conjunto de entrenamiento, adicionalmente calculamos el score para el modelo

In [90]:
model_score = 0
for project in projects_test:
    predict_libraries_from_description(model_train, dictionary_train, similarity_matrix_train, project)
    model_score += project['score']

model_score = model_score/len(projects_test)

El score obtenido es

In [91]:
model_score

0.26552151552151554

# Modelo CountVectorizer

Para cada proyecto nos quedaremos con un 

In [155]:
global_texts_train = [" ".join(project['library']) for project in projects_train]
global_texts_train[:2]

model_countVectorizer = CountVectorizer(min_df=2)

train_data = model_countVectorizer.fit_transform(global_texts_train)

def show_library_similarities(library_similarities):
    for library in sorted(library_similarities, key=library_similarities.get, reverse=True)[:10]:
        print("Library: {0}: - Score: {1}".format(library, library_similarities[library]))

def get_similarities_by_library(train_data, doc, model):
    def cos_distance(v1, v2):
        return 1 - (v1 * v2.transpose()).sum() / (sp.linalg.norm(v1.toarray()) * sp.linalg.norm(v2.toarray()))
    
    new_doc_vectorized = model.transform([doc])[0]
    library_similarities = defaultdict(float)
    for i, doc_vec in enumerate(train_data):
        library_similarities[i] = 1 - cos_distance(doc_vec, new_doc_vectorized)
    return library_similarities


def predict_libraries_from_library(project):
    doc_from_library = " ".join(project['library'])
    project_similarities = get_similarities_by_library(train_data, doc_from_library, model_countVectorizer)
    library_similarities = get_library_similarities(projects, project_similarities, [])
    # Ordernar y pasar a lista, tomamos el mismo número de librerías que las que tiene el proyecto real
    num_lib = len(project['library'])
    return([library for library in sorted(library_similarities, key=library_similarities.get, reverse=True)[:num_lib]])


for project in projects_test: 
    print(project['library'])
    print(predict_libraries_from_library(project))
    print()

['csv', 'ortools']
['bitcoin', 'trafaret']

['argparse', 'pyqrcode', 'logging', 'zbarlight', 'hashlib', 'magic', 'fpdf']
['machines', 'argparse', 're', 'logging', 'time', 'requests', 'hashlib']

['distutils', 'serial', 'time', 'numpy', 'vstruct', 're', 'code', 'select', 'bits', 'elf', 'ctypes']
['machines', 'time', 'tensorflow', 'numpy', 're', 'sklearn', 'argparse', 'unittest', 'keras', 'logging', 'datetime']

['csv', 'redis', 'datetime', 'pymongo', 'time', 'geopy', 'pika', 'requests', 'logging', 'bson', 'geographiclib', 'multiprocessing', 'boto3']
['time', 're', 'datetime', 'logging', 'machines', 'setuptools', 'unittest', 'requests', 'argparse', 'functools', 'numbers', 'hashlib', 'tensorflow']

['numpy', 'scipy', 'lmdb', 'matplotlib']
['machines', 'numpy', 'tensorflow', 'time']

['time', 'arrow', 'requests', 'csv', 'setuptools', 'boto']
['machines', 'time', 're', 'setuptools', 'datetime', 'argparse']

['numpy', 'tensorflow', 'sonnet']
['numpy', 'tensorflow', 'time']

['numpy', 'torch'

In [ ]:
def predict_libraries_from_description(model, dictionary, similarity_matrix, project):  
    get_similarities_by_library(train_data, doc, model)
    library_similarities = get_library_similarities(projects, project_similarities, [])
    # Ordernar y pasar a lista, tomamos el mismo número de librerías que las que tiene el proyecto real
    num_lib = len(project['library'])
    project['library_prediction'] = sorted(library_similarities, key=library_similarities.get, reverse=True)[:num_lib]
    project['score'] = score(project['library'], project['library_prediction'])

.



<span style="color: #12297c; font-family: Arial; font-size: 3em;">Aplicación a un proyecto de ejemplo</span>

Para poner en práctica el sistema de recomendación vamos a usar un proyecto real almacenado en Github. En primer lugar definimos una función que dado un proyecto, lo descargue en local y aplique el mismo procesamiento que hicimos con los proyectos que componen nuestro corpus.

In [ ]:
def proccess_url(git_url):
    def process_python_file(project, file_path):
        def add_to_list(item):
            if not item in library:
                library.append(item)

        library = project['library']
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                match = re.search(LIBRARY_PATTERN, line)
                if match:
                    if match.group(1) is not None:
                        add_to_list(match.group(1))
                    else:
                        add_to_list(match.group(2))
        project['library'] = library

    def process_readme_file(project, file_path):
        with open(file_path, 'r') as f:
            project['readme_txt'] = f.read()
        project['readme_words'] = get_words(project['readme_txt'])

    project = dict()
    os.chdir(ROOT_PATH)
    dir_name = uuid.uuid4().hex
    path = ROOT_PATH + "/" + dir_name

    if not os.path.isdir(path):
        os.system(CLONE_COMMAND.format(git_url, dir_name))
        project['git_url'] = git_url
        project['library'] = []
        for root, dirs, files in os.walk(path):
            for file in files:
                try:
                    if file.endswith(".py"):
                        process_python_file(project, os.path.join(root, file))
                    else:
                        if file.lower().startswith("readme."):
                            process_readme_file(project, os.path.join(root, file))
                except:
                    pass

    return project

Y la invocamos con nuestro projecto de ejemplo

In [ ]:
sample_project = proccess_url("https://github.com/yazquez/programmable-agents_tensorflow.git")

Podemos ver las librerías que está usando

In [ ]:
sample_project['library'][:3]

Así como su descripción (fichero **readme**)

In [ ]:
sample_project['readme_txt'][:2000]

Y la descomposición de dicho fichero en sus palabras significativas

In [ ]:
sample_project['readme_words'][:7]

Si recapitulamos, tenemos las siguientes funciones definidas:

- **get_similarities_by_description**: Devuelve los proyectos similares en funcíon de sus descripciones
- **get_similarities_by_library**: Devuelve los proyectos similares en funcíon de las librerías que se usa
- **get_library_similarities(project_similarities)**: Devuelve las librerías comunes usando una lista de proyectos


Las aplicamos a nuestro proyecto de ejemplo

Empezamos obteniendo los proyectos más similares al del ejemplo usando las descripciones y a partir de estos proyectos obtenemos las librerías

In [ ]:
project_similarities_by_description = get_similarities_by_description(model, dictionary, similarity_matrix, sample_project['readme_words'])

In [ ]:
library_similarities_by_description = get_library_similarities(projects, project_similarities_by_description, sample_project['library'])

Mostramos los 10 proyectos más similares

In [ ]:
show_project_similarities(projects, project_similarities_by_library)

Hacemos lo mismo usando las librerías del proyecto como medida de similitud

In [ ]:
project_similarities_by_library = get_similarities_by_library(X_train_data, " ".join(sample_project['library']), countVectorizer)

In [ ]:
library_similarities_by_libraries = get_library_similarities(projects, project_similarities_by_library, sample_project['library'])

Mostramos los 10 proyectos más similares

In [ ]:
show_project_similarities(projects, project_similarities_by_library)

Finalmente mostramos el resultado, o sea las librerías recomendadas

**Librerías recomendadas en función de las descripciones**

In [ ]:
show_library_similarities(library_similarities_by_description)

**Librerías recomendadas en función de las librerías comunes**

In [ ]:
show_library_similarities(library_similarities_by_libraries)